In [1]:
import requests
from requests.exceptions import ConnectionError
from time import sleep
import json
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from pprint import pprint
from openpyxl import load_workbook
from openpyxl import Workbook
from pprint import pprint
from datetime import date, timedelta, datetime
from time import time
import sys

In [2]:
import httplib2
import argparse
from apiclient import discovery
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage
import gspread
from oauth2client.service_account import ServiceAccountCredentials

#### Google Sheets API auth

In [3]:
SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
CLIENT_SECRET_FILE = 'sheets-client-secret-grants.json'
APPLICATION_NAME = 'Google Sheets API Report'

credential_path = 'sheets.googleapis.com-report.json'

store = Storage(credential_path)
credentials = store.get()
parser = argparse.ArgumentParser(formatter_class=argparse.RawDescriptionHelpFormatter, 
                                 parents=[tools.argparser])
flags = parser.parse_args([])

if not credentials or credentials.invalid:
    flow = client.flow_from_clientsecrets(r'.../sheets-client-secret-grants.json', 
                                          SCOPES)
    flow.user_agent = APPLICATION_NAME
    if flags:
        credentials = tools.run_flow(flow, store,flags)
    print('Storing credentials to ' + credential_path)

#### Google Sheets services

In [4]:
http = credentials.authorize(httplib2.Http())
discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?version=v4')
service = discovery.build('sheets', 'v4', http = http, discoveryServiceUrl=discoveryUrl)

In [5]:
token = '...'

spreadsheet_id = '1eVfRLLTEfUfTlJrnXZYZfRSiWZYWtOQ7A4sUO68PMvg' # spreadsheet id

In [6]:
if sys.version_info < (3,):
    def u(x):
        try:
            return x.encode("utf8")
        except UnicodeDecodeError:
            return x
else:
    def u(x):
        if isinstance(x, bytes):
            return x.decode('utf8')
        else:
            return x

# input
AgencyClientsURL = 'https://api.direct.yandex.com/json/v5/agencyclients'

ReportsURL = 'https://api.direct.yandex.com/json/v5/reports'

# OAuth
token = token

# AgencyClients
headers = {
           # OAuth-токен
           "Authorization": "Bearer " + token,
           "Accept-Language": "ru"
           }

AgencyClientsBody = {
    "method": "get",
    "params": {
        "SelectionCriteria": {
            "Archived": "NO"   # active clients only: no
        },
        "FieldNames": ["Login"],
        "Page": {
            "Limit": 10000,
            "Offset": 0
        }
    }
}

# AgencyClients output
HasAllClientLoginsReceived = False
ClientList = []

while not HasAllClientLoginsReceived:
    ClientsResult = requests.post(AgencyClientsURL, json.dumps(AgencyClientsBody), headers=headers).json()
    for Client in ClientsResult['result']['Clients']:
        ClientList.append(Client["Login"])
    if ClientsResult['result'].get("LimitedBy", False):
        AgencyClientsBody['Page']['Offset'] = ClientsResult['result']["LimitedBy"]
    else:
        HasAllClientLoginsReceived = True

# Reports input
body = {
    "params": {
        "SelectionCriteria": {},
        "FieldNames": [
            "Impressions",
            "Clicks",
            "Cost"
        ],
        "ReportName": u("ACCOUNT_PERFORMANCE"),
        "ReportType": "ACCOUNT_PERFORMANCE_REPORT",
        "DateRangeType": "THIS_MONTH", # 'AUTO', 'THIS_MONTH', 'LAST_MONTH' or 'CUSTOM_DATE'
        "Format": "TSV",
        "IncludeVAT": "NO", # NDS
        "IncludeDiscount": "NO"
    }
}

# result data
resultcsv = "Login;Impressions;Clicks;Costs\n"

# headers
headers['skipReportHeader'] = "true"
headers['skipColumnHeader'] = "true"
headers['skipReportSummary'] = "true"
headers['returnMoneyInMicros'] = "false"

# Reports output
for Client in ClientList:
    headers['Client-Login'] = Client
    requestBody = json.dumps(body, indent=4)
    
    while True:
        try:
            req = requests.post(ReportsURL, requestBody, headers=headers)
            req.encoding = 'utf-8'
            if req.status_code == 400:
                print("Параметры запроса указаны неверно или достугнут лимит отчетов в очереди")
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                print("JSON-код запроса: {}".format(u(body)))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break
            elif req.status_code == 200:
                print("Отчет для аккаунта {} создан успешно".format(str(Client)))
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                if req.text != "":
                    tempresult = req.text.split('\t')
                    resultcsv += "{};{};{};{}\n".format(Client, tempresult[0], tempresult[1], str(tempresult[2]).replace('.', ','))
                else:
                    resultcsv += "{};0;0;0\n".format(Client)
                break
            elif req.status_code == 201:
                print("Отчет для аккаунта {} успешно поставлен в очередь в режиме offline".format(str(Client)))
                retryIn = int(req.headers.get("retryIn", 60))
                print("Повторная отправка запроса через {} секунд".format(retryIn))
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                sleep(retryIn)
            elif req.status_code == 202:
                print("Отчет формируется в режиме офлайн".format(str(Client)))
                retryIn = int(req.headers.get("retryIn", 60))
                print("Повторная отправка запроса через {} секунд".format(retryIn))
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                sleep(retryIn)
            elif req.status_code == 500:
                print("При формировании отчета произошла ошибка. Пожалуйста, попробуйте повторить запрос позднее.")
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break
            elif req.status_code == 502:
                print("Время формирования отчета превысило серверное ограничение.")
                print(
                    "Пожалуйста, попробуйте изменить параметры запроса - уменьшить период и количество запрашиваемых данных.")
                print("JSON-код запроса: {}".format(body))
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break
            else:
                print("Произошла непредвиденная ошибка")
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                print("JSON-код запроса: {}".format(body))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break

        except ConnectionError:
            print("Произошла ошибка соединения с сервером API")
            break

        except:
            print("Произошла непредвиденная ошибка")
            break

print("Создание отчетов для аккаунтов завершено")

"""# Создание и запись файла
filename = "AccountsPerfomanceReport_{}.csv".format(str(time()))
resultfile = open(filename, 'w+')
resultfile.write(resultcsv)
resultfile.close()

print("Результат записан в файл {}".format(filename))"""

Отчет для аккаунта zlatapechkaru создан успешно
RequestId: 4603484569976803120
Отчет для аккаунта nakolesahmira создан успешно
RequestId: 4603485446334607023
Отчет для аккаунта sterlingcooper создан успешно
RequestId: 4603486332142509837
Отчет для аккаунта advantagroupru создан успешно
RequestId: 4603487221225820769
Отчет для аккаунта fanburg создан успешно
RequestId: 4603488161835214491
Отчет для аккаунта zapovednik-zapovednik создан успешно
RequestId: 4603489043813669006
Отчет для аккаунта nakolesahmira-farkopy создан успешно
RequestId: 4603489755032428760
Отчет для аккаунта nakolesahmira-gruzovie-shiny создан успешно
RequestId: 4603490723418991720
Отчет для аккаунта promoshift-trusishka создан успешно
RequestId: 4603491474831106019
Отчет для аккаунта zapovednik-tyumen создан успешно
RequestId: 4603492431304079670
Отчет для аккаунта b-bestwatch создан успешно
RequestId: 4603493611419084256
Отчет для аккаунта c-csoft создан успешно
RequestId: 4603494139749407616
Отчет для аккаунта buy

'# Создание и запись файла\nfilename = "AccountsPerfomanceReport_{}.csv".format(str(time()))\nresultfile = open(filename, \'w+\')\nresultfile.write(resultcsv)\nresultfile.close()\n\nprint("Результат записан в файл {}".format(filename))'

#### Date generation

In [7]:
month = str(datetime.now().month)

if datetime.now().month < 10: # Для месяцев, порядковый номер меньше 10 добавляем "ноль" для корректного отображения
    month = '-0' + month
else:
    month = '-' + month
    
start_date = str(datetime.now().year) + month + '-01'
end_date = datetime.today().strftime('%Y-%m-%d')

In [9]:
result = []

for i in resultcsv.split():
    result.append(i.split(';'))

for i in result:
    i.insert(0, start_date)
    i.insert(1, end_date)
    i.pop(3)

result.pop(0)

['2023-01-01', '2023-01-12', 'Login', 'Clicks', 'Costs']

#### Data to the spreadsheet

In [10]:
body = {'values': result}

In [12]:
service.spreadsheets().values().clear(spreadsheetId=spreadsheet_id, 
                                      range='Лист1'+'!E2:I', 
                                      body={}).execute()

service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range='Лист1'+'!E2:I', 
                                       valueInputOption='USER_ENTERED', 
                                       body=body).execute()

print('https://docs.google.com/spreadsheets/d/1eVfRLLTEfUfTlJrnXZYZfRSiWZYWtOQ7A4sUO68PMvg/edit#gid=0')

print('Google Data Studio report template: https://datastudio.google.com/reporting/a1cbd283-a91a-45b2-bc91-b6d17fe88d5a')

https://docs.google.com/spreadsheets/d/1eVfRLLTEfUfTlJrnXZYZfRSiWZYWtOQ7A4sUO68PMvg/edit#gid=0
Google Data Studio report template: https://datastudio.google.com/reporting/a1cbd283-a91a-45b2-bc91-b6d17fe88d5a


#### Data source and dashboard template

Spread sheet using as a data source for Google Data Studio dashboard template: 
* https://docs.google.com/spreadsheets/d/1eVfRLLTEfUfTlJrnXZYZfRSiWZYWtOQ7A4sUO68PMvg/edit#gid=0

Google Data Studio report template: 
* https://datastudio.google.com/reporting/a1cbd283-a91a-45b2-bc91-b6d17fe88d5a